In [99]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D,AveragePooling2D
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import DenseNet121

In [100]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [101]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [102]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [103]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [104]:
weights_Add="/kaggle/input/weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
base_model = MobileNetV2(weights=weights_Add, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [105]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(5, activation='softmax')  
])

In [106]:
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 62720)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │     4,014,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_29 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,272,453 (23.93 MB)

 Trainable params: 4,014,469 (15.31 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [107]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 428ms/step - accuracy: 0.6771 - loss: 5.5319 - val_accuracy: 0.8890 - val_loss: 0.4311
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 342ms/step - accuracy: 0.8806 - loss: 0.5460 - val_accuracy: 0.8786 - val_loss: 0.4759
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 31s 354ms/step - accuracy: 0.8933 - loss: 0.3827 - val_accuracy: 0.8454 - val_loss: 0.7703
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 31s 348ms/step - accuracy: 0.8879 - loss: 0.5272 - val_accuracy: 0.8589 - val_loss: 0.7676
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 31s 354ms/step - accuracy: 0.8996 - loss: 0.4846 - val_accuracy: 0.8983 - val_loss: 0.6140
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 370ms/step - accuracy: 0.9061 - loss: 0.4554 - val_accuracy: 0.8983 - val_loss: 0.6177
